### Getting Started

SciPy is the application level library for scientific computing in Python. It provides an assortment of tools for linear algebra, optimization, interpolation, statistics, and signal processing.

The SciPy source documentation can be found here:
<br>https://www.scipy.org/

A quickstart tutorial can be found here:
<br>https://docs.scipy.org/doc/scipy/reference/tutorial/index.html

Documentation on optimization and curve-fitting can be found here:
<br>https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

Let's start by importing elements of the scipy.optimize package. We'll also import the scipy.interpolate package with the "ipt" alias, and the numpy package with the np alias. We'll finish by importing the pyplot package from matplotlib with the "plt" alias.

In [ ]:
import scipy.interpolate as itp
from scipy.optimize import fsolve, curve_fit, minimize
import numpy as np
import matplotlib.pyplot as plt
from evaluation import day3_eval1, day3_eval2

%matplotlib inline

In [ ]:
#Let's inspect the optimize module
from scipy import optimize
optimize.minimize?

In [ ]:
#Let's inspect the interpolate module
itp?

### Solving Systems of Equations

SciPy offers a convenience function for solving systems of equations. Consider the following system of equations:

$x + y^{2} - 4 = 0$
<br>$e^{x} + xy - 3 = 0$

We will use the <i> fsolve </i> function to simultaneously solve for x and y.

In [ ]:
# Create a system of equations function that accepts the equation terms as 
# parameters (p) and returns the system results as a tuple.

def equations(p):
    """
    param p: a tuple containing values for x and y
    
    returns: a tuple of equation outputs given x and y
    """
    x, y = p
    
    equation1 = x + y**2 - 4
    equation2 = np.exp(x) + x * y - 3
    
    return (equation1, equation2)

print(equations((1, 1)))

In [ ]:
# Solve for system terms given a starting guess (p0) and the system of equations function
p0 = (1, 1)
x, y =  fsolve(equations, p0)
print(x,y)

Exercise 1. Consider the following system of equations:

$3x^{2} - 4y + xz = 2$
<br>$xy - 5yx + 3x = 2$
<br>$2e^{xy/z} + 2yz + z = 4$

Use the <i> fsolve </i> function to simultaneously solve for x, y, and z. Submit your answers to day3_eval1() for asssement.

In [ ]:
def eqs(p):
    
    x, y, z = p
    
    eq1 = 3 * x ** 2 - 4 * y + x * z - 2
    eq2 = x * y - 5 * y * x + 3 * x - 2
    eq3 = 2 * np.exp(x * y / z) + 2 * y * z + z - 4
    
    return (eq1, eq2, eq3)

p0 = (1, 1, 1)
x, y, z =  fsolve(eqs, p0)
print(x,y,z)

day3_eval1(np.array([x, y, z]))

In [ ]:
fsolve?

### Interpolation

Interpolation functions are provided as part of the SciPy library. Interpolation is used to fill in missing data in a data set in a principled way. It is a good tool for rudimentary upsampling. It's also a good way to visualize data and infer functions for curve fitting.

Let's get started with a simple 1-dimensional linear interpolation.

In [ ]:
# Actual signal generation function
def f(x):
    return np.sin(x * np.pi)

x = np.linspace(0,10,250)
y = f(x)

plt.plot(x, y, 'g-', label='actual')
plt.legend(loc='upper right');

In [ ]:
# Data sample from the actual signal
x_data = np.linspace(0,10,50)
y_data = f(x_data)

plt.plot(x_data,y_data,'c.',markersize=15, label='sample')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Create the interpolator and generate interpolated data over a sample range x
f_itp = itp.interp1d(x_data, y_data)

x_new = np.linspace(1,9,100)
y_new = f_itp(x_new)

plt.plot(x, y, 'g-', label='actual')
plt.plot(x_data,y_data,'c.',markersize=15, label='sample')
plt.plot(x_new, y_new, 'r.', label='1-D interpolation')
plt.legend(loc='upper right')
plt.show()

You can see that the linear interpolation can be a crude approximation for highly non-linear functions. A more sophisticated approach is to use a polynomial interpolator. Below is an example of a CubicSpline. 

In [ ]:
# Polynomial 1D
f_itp = itp.CubicSpline(x_data,y_data)

x_new = np.linspace(1,9,100)
y_new = f_itp(x_new)

plt.plot(x, y, 'g-', label='actual')
plt.plot(x_data,y_data,'c.',markersize=15, label='sample')
plt.plot(x_new, y_new, 'r.', label='1-D interpolation')
plt.legend(loc='upper right')
plt.show()


### Curve fitting

Fitting experimental data to known functions is a common practice in science and engineering. The optimize package from SciPy provides a curve fitting routine. Below is an example.

In [ ]:
# We'll define a function that accepts x and equation parameters and returns f(x).
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50)
np.random.seed(1729)
y = func(xdata, 2.5, 1.3, 0.5)

ydata = y + np.random.normal(0,0.1, size=xdata.size)

plt.plot(xdata, ydata, 'b.', label='data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

Now we'll do the curvefit. Note that the curve_fit function returns both a parameters and a convariance matrix. We will use the covariance matrix to provide an estimate on the parameter uncertainty.

In [ ]:
curve_fit?

In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, bounds=[[0,0,0],[5,5,5]])

err = np.sqrt(np.diag(pcov)**2)

print('a: %.6f'%popt[0], '+/- %.6f'%err[0])
print('b: %.6f'%popt[1], '+/- %.6f'%err[1])
print('c: %.6f'%popt[2], '+/- %.6f'%err[2])


In [ ]:
#Analyze the fit.
plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.plot(xdata, ydata, 'b.', label='data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

Exercise 2. The daily average temperatures from Albany are provided with a code snippet to load in the following cell.

Fit the data to the following function:

<center>$f(t)=a*sin(2πt+b) + c$</center>
 
where  t  is the time in <b><i>years</i></b>. Make a plot of the data and the best-fit model.

Submit the best-fit values of the parameters to day3_eval2() to see how you did. 

In [ ]:
# The following code reads in the data from a csv file
day, temperature = np.loadtxt('albany_weather_data.csv', unpack=True, delimiter=',')

plt.plot(day, temperature, 'c.')
plt.title('Daily Mean Temp, Albany, NY - 2017')
plt.xlabel('Time (d)')
plt.ylabel('Temperature (C)')
plt.show()


In [ ]:
def f(t, a, b, c):
    return a * np.sin(2 * np.pi * t + b) + c

t_norm = day / 365

popt, pcov = curve_fit(f, t_norm, temperature, p0=[-10,1,10])

y_pred = f(t_norm, *popt)

plt.plot(day, temperature, 'c.')
plt.plot(day, y_pred, 'r-')
plt.title('Daily Mean Temp, Albany, NY - 2017')
plt.xlabel('Time (d)')
plt.ylabel('Temperature (C)')
plt.show()

print(popt)

day3_eval2(popt)



### Unconstrained Minimization

To demonstrate the available minimizers in the SciPy optimizer package we will do a simple least-squares regression on the Albany dataset. The default minimizer for unconstrained problems without bounds is the Nelder-Mead simplex method. Additional information on the available solvers can be found here:
<br> https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

In [ ]:
# First lets define our target, or fit, function f(t)
def f(t, a, b, c):
    return a * np.sin(2 * np.pi * t / 365 + b) + c

# Then we'll define our cost function as a function of the target function parameters
def mean_squared_error(p):
    return ((temperature - f(day, p[0], p[1], p[2])) ** 2).mean()

p0 = np.ones(3)
mean_squared_error(p0)

Note that the minimizer provides the parameter matrix (res.x) and the inverse Hessian matrix (res.hess_inv). We will use the inverse Hessian to compute parameter uncertainties.

In [ ]:
minimize?

In [ ]:
res = minimize(mean_squared_error,p0)
print(res)
err = np.sqrt(np.diag(res.hess_inv) ** 2)

print('\na: %.6f'%res.x[0], '+/- %.6f'%err[0])
print('b: %.6f'%res.x[1], '+/- %.6f'%err[1])
print('c: %.6f'%res.x[2], '+/- %.6f'%err[2])

In [ ]:
# Analyze the results
temp_fit1 = f(day, *res.x)

plt.plot(day, temperature, 'c.', label='data')
plt.plot(day, temp_fit1, 'r-', label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(res.x))

plt.title('Daily Mean Temp, Albany, NY - 2017')
plt.xlabel('time (d)')
plt.ylabel('teperature (C)')
plt.legend()
plt.show()



### Constrained Minimization

In [ ]:
def cons(x):
    # x > 15
    # x - 15 > 0
    return x[0] - 15.0

x0 = np.ones(3) * 4
constraints = [{'type': 'ineq', 'fun': cons}]

res = minimize(mean_squared_error, x0, constraints=constraints)
temp_fit2 = f(day, *res.x)

plt.plot(day, temperature, 'c.', label='data')
plt.plot(day, temp_fit1, 'b--', label='unconstrained fit')
plt.plot(day, temp_fit2, 'g-', label='constrained fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(res.x))

plt.title('Daily Mean Temp, Albany, NY - 2017')
plt.xlabel('time (d)')
plt.ylabel('teperature (C)')
plt.legend()
plt.show()


Exersize 3. Using the constrained minimizer above, add constraints on b & c parameters and plot your results below.

In [ ]:
def cons(x):
    # x > 15
    # x - 15 > 0
    return x[0] - 15.0

def cons1(x):
    # x > 5
    # x - 5 > 0
    return x[1] - 5

def cons2(x):
    # x > 6
    # x - 6 > 0
    return x[2] - 6

x0 = np.ones(3) * 4
constraints = [{'type': 'ineq', 'fun': cons}, 
               {'type': 'ineq', 'fun': cons1},
               {'type': 'ineq', 'fun': cons2}]

res = minimize(mean_squared_error, x0, method='COBYLA',constraints=constraints)
temp_fit2 = f(day, *res.x)

plt.plot(day, temperature, 'c.', label='data')
plt.plot(day, temp_fit1, 'b--', label='unconstrained fit')
plt.plot(day, temp_fit2, 'g-', label='constrained fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(res.x))

plt.title('Daily Mean Temp, Albany, NY - 2017')
plt.xlabel('time (d)')
plt.ylabel('teperature (C)')
plt.legend()
plt.show()